In [1]:
from absl import app
from absl import flags
import apache_beam as beam
from proto.stu3 import google_extensions_pb2
from proto.stu3 import resources_pb2
from py.google.fhir.labels import encounter
from py.google.fhir.labels import label


/usr/local/envs/py2env/lib/python2.7/site-packages/ipykernel/__main__.py:4: ImportWarning: Not importing directory '/usr/local/fhir/proto': missing __init__.py


In [2]:
@beam.typehints.with_input_types(resources_pb2.Bundle)
@beam.typehints.with_output_types(google_extensions_pb2.EventLabel)
class LengthOfStayRangeLabelAt24HoursFn(beam.DoFn):
  """Converts Bundle into length of stay range at 24 hours label.

    Cohort: inpatient encounter that is longer than 24 hours
    Trigger point: 24 hours after admission
    Label: multi-label for length of stay ranges, see label.py for detail
  """

  def process(self, bundle):
    """Iterate through bundle and yield label.

    Args:
      bundle: input stu3.Bundle proto
    Yields:
      stu3.EventLabel proto.
    """
    patient = encounter.GetPatient(bundle)
    if patient is not None:
      # Cohort: inpatient encounter > 24 hours.
      for enc in encounter.Inpatient24HrEncounters(bundle):
        for one_label in label.LengthOfStayRangeAt24Hours(patient, enc):
          yield one_label



## Initialize pipeline variables

In [3]:
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import PipelineOptions

from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.metrics import Metrics
from apache_beam.metrics.metric import MetricsFilter

import apache_beam as beam
import re

options = PipelineOptions()
google_cloud_options = options.view_as(GoogleCloudOptions)
google_cloud_options.project = 'de-test-224618'
google_cloud_options.job_name = 'job1'
google_cloud_options.staging_location = 'gs://healthedatalab/staging'
google_cloud_options.temp_location = 'gs://healthedatalab/temp'
options.view_as(StandardOptions).runner = 'DirectRunner'


## Initalize the beam job

In [4]:

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
p = beam.Pipeline(options=options)

bundles = p | 'read' >> beam.io.ReadFromTFRecord(
    'gs://healthedatalab/bundle/test_bundle.tfrecord-00000-of-00001', coder=beam.coders.ProtoCoder(resources_pb2.Bundle))
    
labels = bundles | 'BundleToLabel' >> beam.ParDo(
    LengthOfStayRangeLabelAt24HoursFn())
_ = labels | beam.io.WriteToTFRecord(
    'gs://healthedatalab/labeldata/test-label',
    coder=beam.coders.ProtoCoder(google_extensions_pb2.EventLabel))



/usr/local/envs/py2env/lib/python2.7/site-packages/oauth2client/contrib/gce.py:99: UserWarning: You have requested explicit scopes to be used with a GCE service account.
Using this argument will have no effect on the actual scopes for tokens
requested. These scopes are set at VM instance creation time and
can't be overridden in the request.

  warnings.warn(_SCOPES_WARNING)
I0109 04:06:16.353255 139734587995904 client.py:614] Attempting refresh to obtain initial access_token
I0109 04:06:16.603811 139734587995904 client.py:614] Attempting refresh to obtain initial access_token


## Run the transform

In [5]:
   p.run().wait_until_finish()

I0109 04:06:49.158493 139734587995904 fn_api_runner.py:844] ==================== <function annotate_downstream_side_inputs at 0x7f1641b8e668> ====================
I0109 04:06:49.161040 139734587995904 fn_api_runner.py:844] ==================== <function fix_side_input_pcoll_coders at 0x7f1641b8e578> ====================
I0109 04:06:49.164942 139734587995904 fn_api_runner.py:844] ==================== <function lift_combiners at 0x7f1641b8e230> ====================
I0109 04:06:49.166904 139734587995904 fn_api_runner.py:844] ==================== <function expand_gbk at 0x7f1641b8e9b0> ====================
I0109 04:06:49.171428 139734587995904 fn_api_runner.py:844] ==================== <function sink_flattens at 0x7f1641b8e8c0> ====================
I0109 04:06:49.173270 139734587995904 fn_api_runner.py:844] ==================== <function greedily_fuse at 0x7f1641b8eaa0> ====================
I0109 04:06:49.177407 139734587995904 fn_api_runner.py:844] ==================== <function sort_stag

I0109 04:06:50.054879 139734587995904 bundle_processor.py:303] finish <DataOutputOperation ref_PCollection_PCollection_11/Write >
I0109 04:06:50.064006 139734587995904 fn_api_runner.py:912] Running ((ref_PCollection_PCollection_3/Read)+(ref_AppliedPTransform_WriteToTFRecord/Write/WriteImpl/PreFinalize_19))+(ref_PCollection_PCollection_12/Write)
I0109 04:06:50.072561 139734587995904 bundle_processor.py:291] start <DataOutputOperation ref_PCollection_PCollection_12/Write >
I0109 04:06:50.074151 139734587995904 bundle_processor.py:291] start <DoOperation WriteToTFRecord/Write/WriteImpl/PreFinalize output_tags=['out']>
I0109 04:06:50.077641 139734587995904 bundle_processor.py:291] start <DataInputOperation ref_PCollection_PCollection_3/Read receivers=[ConsumerSet[ref_PCollection_PCollection_3/Read.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
I0109 04:06:50.083122 139734587995904 gcsio.py:446] Starting the size estimation of the input
I0109 04:06:50.086520 139734

'DONE'

YAY!!